## Rollups:
  - [April 11, 2017—KB4015550 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4015550/windows-8-1-windows-server-2012-r2-update-kb4015550/)
  - [May 9, 2017—KB4019215 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4019215/windows-8-update-kb4019215/)
  - [June 13, 2017—KB4022726 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4022726/windows-8-update-kb4022726/)
  - [July 11, 2017—KB4025336 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4025336/windows-8-update-kb4025336/)
  - [August 8, 2017—KB4034681 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4034681/windows-8-1-windows-server-2012-r2-update-kb4034681/)
  - [September 12, 2017—KB4038792 (OS Build Monthly Rollup)](https://support.microsoft.com/en-us/help/4038792/windows-8-1-update-kb4038792/)
  - [October 10, 2017—KB4041693 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4041693/windows-81-update-kb4041693/)
  - [November 14, 2017—KB4048958 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4048958/windows-81-update-kb4048958/)
  - [December 12, 2017—KB4054519 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4054519/windows-81-update-kb4054519/)
  - [January 8, 2018—KB4056895 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4056895/windows-81-update-kb4056895/)
  - [February 13, 2018—KB4074594 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4074594/windows-81-update-kb4074594/)
  - [March 13, 2018—KB4088876 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4088876/windows-81-update-kb4088876/)
  - [April 10, 2018—KB4093114 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4093114/windows-81-update-kb4093114/)
  - [May 8, 2018—KB4103725 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4103725/windows-81-update-kb4103725/)
  - [June 12, 2018—KB4284815 (Monthly Rollup)](https://support.microsoft.com/en-us/help/4284815/windows-81-update-kb4284815/)

In [224]:
rollup_kbno = [
    4015550,
    4019215,
    4022726,
    4025336,
    4034681,
    4038792,
    4041693,
    4048958,
    4054519,
    4056895,
    4074594,
    4088876,
    4093114,
    4103725,
    4284815,
]

In [230]:
from scrapy.selector import Selector
import requests
import re
import json

In [226]:
def hotfix_info(kb):
    kb = re.sub('^kb', '', str(kb).lower().strip())
    assert re.match(r'^\d+$', kb)

    sess = requests.Session()
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0'
    sess.headers.update({'User-Agent': user_agent})
    
    url1 = 'http://www.catalog.update.microsoft.com/Search.aspx?q=KB%s' % kb
    page1 = sess.get(url1)
    sel1 = Selector(text=page1.text)
    
    els = sel1.css('#tableContainer table')
    xpath = '''tr[contains(td[3], "Windows 8.1") and contains(td[2], "8.1 for x64-")]'''
    osel = els.xpath(xpath)
    assert len(osel) == 1

    fdesc = osel.xpath('./td[2]/a/text()').extract_first().strip()
    assert fdesc

    js = osel.xpath('./td[2]/a/@onclick').extract_first()
    mat = re.match(r'goToDetails\("([-a-z0-9]+)"\);', js)
    assert mat is not None
    uid = mat.group(1)

    url2 = 'http://www.catalog.update.microsoft.com/ScopedViewInline.aspx?updateid=%s' % uid
    page2 = sess.get(url2)
    sel2 = Selector(text=page2.text)
    
    els = sel2.css("#supersededbyInfo div a ::text").extract()
    supedby = [re.search(r'\(KB(\d+)\)', txt).group(1) for txt in els]
    supedby = sorted(set(supedby))
    if not supedby:
        print('warning: kb %s not superseded' % kb)
    
    els = sel2.css("#supersedesInfo div ::text").extract()
    supes = [re.search(r'\(KB(\d+)\)', txt).group(1) for txt in els]
    supes = sorted(set(supes))
    assert supes
    
    url3 = 'http://www.catalog.update.microsoft.com/DownloadDialog.aspx'
    updateIDs = '[{"size":0,"languages":"","uidInfo":"%s","updateID":"%s"}]' % (uid, uid)
    data3 = {
        'contentImport': '',
        'portNumber': '',
        'serverName': '',
        'sku': '',
        'ssl': '',
        'updateIDs': updateIDs,
        'updateIDsBlockedForImport': '',
        'version': '',
        'wsusApiPresent': '',
    }
    page3 = sess.post(url3, data=data3)
    
    regex = r'''downloadInformation\[\d+\]\.files\[\d+\]\.url\s*=\s*'([^']+)';'''
    file_urls = re.findall(regex, page3.text)
    assert len(file_urls) == 1
    furl = file_urls[0]
    assert furl.startswith('http://')
    assert furl.endswith('.msu')

    url4 = 'https://support.microsoft.com/en-us/help/%s/' % kb
    page4 = requests.get(url4)
    kburl = page4.url
    txt4 = re.findall(r'''\s+"heading":\s+"([^"]+)",''', page4.text)
    assert len(txt4) == 1
    kbtitle = txt4[0].strip()
    assert kbtitle
    
    res= {
        'kbno': kb,
        'kburl': kburl,
        'kbtitle': kbtitle,
        'caturl': url1,
        'supedby': ' '.join(supedby),
        'supes': ' '.join(supes),
        'fdesc': fdesc,
        'furl': furl,
    }

    return res

In [227]:
rollups_info = [hotfix_info(no) for no in rollup_kbno]

In [228]:
rollups_info

[{'caturl': 'http://www.catalog.update.microsoft.com/Search.aspx?q=KB4015550',
  'fdesc': 'April, 2017 Security Monthly Quality Rollup for Windows 8.1 for x64-based Systems (KB4015550)',
  'furl': 'http://download.windowsupdate.com/c/msdownload/update/software/secu/2017/04/windows8.1-kb4015550-x64_516ecbc130cb85fe3ae74f04c9f2cc791b669012.msu',
  'kbno': '4015550',
  'kbtitle': 'April 11, 2017—KB4015550 (Monthly Rollup)',
  'kburl': 'https://support.microsoft.com/en-us/help/4015550/windows-8-1-windows-server-2012-r2-update-kb4015550/',
  'supedby': '4019215',
  'supes': '2876331 2893984 2894853 2900986 2913760 2916036 2918614 2926765 2930275 2939153 2939576 2957189 2957390 2961858 2962123 2964718 2964736 2965155 2965161 2965788 2966034 2966631 2966870 2976536 2989542 2993651 2993958 2998527 2999226 3000061 3000988 3002885 3003743 3004394 3006137 3006958 3008188 3008627 3013126 3013455 3019215 3019868 3023562 3025417 3029803 3029944 3031432 3032323 3033889 3034344 3035017 3035131 3035132

In [234]:
with open('ms-rollups.json', 'w', encoding='utf-8') as fp:
    json.dump(rollups_info, fp, ensure_ascii=False, indent=2, sort_keys=True)

In [ ]:
markdown = [" - [%(kbtitle)s](%(kburl)s)\n" % info for info in rollups_info]
print(*markdown)